# Biostat 257 Homework 4

**Due May 28 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.6.0)
  CPU: Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, westmere)


In this assignment, we continue with the linear mixed effects model (LMM) considered in HW2
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T,
$$
Because the variance component parameter $\boldsymbol{\Sigma}$ has to be positive semidefinite, we use its Cholesky factor $\mathbf{L}$ as optimization variable. 

Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2).
$$
In this assignment, we use the nonlinear programming (NLP) approach for optimization. In HW5, we will derive a EM (expectation-maximization) algorithm for the same problem. 

IN HW2 we know how to evaluate it efficiently. 
So we need a function to calculate / evaluate the gradient

In [5]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions, Ipopt, LinearAlgebra, 
MathProgBase, MixedModels, NLopt, Random, RCall, Revise

## Q1. (Optional, 30 bonus pts) Derivatives

NLP optimization solvers expect users to provide at least a function for evaluating objective value. If users can provide further information such as gradient and Hessian, the NLP solvers will be more stable and converge faster. Automatic differentiation tools are becoming more powerful but cannot apply to all problems yet.

1. Show that the gradient of $\ell_i$ is
\begin{eqnarray*}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i, \\
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i, \\
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L},
\end{eqnarray*}
where $\mathbf{r}_i = \mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}$. 

Recall L is the cholesky factor of $\Sigma$. 
(He is giving us the derivation directly)

2. Derive the observed information matrix and the expected (Fisher) information matrix.

If you need a refresher on multivariate calculus, my [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) may be helpful.

## Q2. (20 pts) Objective and gradient evaluator for a single datum

We expand the code from HW2 to evaluate both objective and gradient. I provide my code for HW2 below as a starting point. You do _not_ have to use this code. If your come up faster code, that's even better. 


He calculates the quadratic part of the log likelihood this way:
$$\frac{-1}{2}(y_i - X_i\beta)^T(\sigma^2I_{n_i} + Z_iLL^TZ_i^T)^{-1}(y_i - X_i\beta) = \\
\frac{-1}{2\sigma^2}\big[(y_i - X_i\beta)^T(y_i - X_i\beta) - (y_i - X_i\beta)^TZ_iLM^{-1}L^TZ_i(y_i - X_i\beta)\big]$$
Where $M^{-1} = (\sigma^2I_q + L^TZ_i^TZ_iL)^{-1}$.

$\Omega_i^{-1} = \frac{1}{\sigma^2}\big[I_{n_i} - Z_iL(\sigma^2I_q + L^TZ_i^TZ_iL)^{-1}L^TZ_i^T\big] = \frac{1}{\sigma^2}\big[I_{n_i} - Z_iLM^{-1}L^TZ_i\big]$

using woodbury with $U = Z_iL, B = \frac{1}{\sigma^2}I_q, V^T = L^TZ_i^T$. 

He stores Cholesky of $M$ in `storage_qq`. Recall that $M = LL^T = U^TU$

He treats  $M^{-1}L^TZ_i(y_i - X_i\beta)$ as a triangle solve with $M = U^TU$. Then $U \hspace{.2cm} / \hspace{.2cm} L^TZ_i(y_i - X_i\beta)$ is actually one half of $(y_i - X_i\beta)^TZ_iLM^{-1}L^TZ_i(y_i - X_i\beta)$. Then we can just treat it as a dot product! He stores the triangle solve in `storage_q`.


In [2]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    ltztzl     :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    ltztzl     = similar(ztz)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, storage_p, storage_q, storage_q2,  
        xtx, ztx, ztz, storage_qq, ltztzl)
end

LmmObs

`logl!(obs::LmmObs, β, L, σ², needgrad=false)`

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ²` are filled 
with the corresponding gradient. 

This is how I am computing the gradients. 
$$\nabla_{\boldsymbol{\beta}} = \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i$$
$$ = X_i^T\big[\frac{1}{\sigma^2}(I_{n_i} - Z_iLM^{-1}L^TZ_i^T)\big](y-X_i\beta)$$
$$ = \frac{1}{\sigma^2}\big[X_i^T(y-X_i\beta) - X^TZ_iL U \hspace{.1cm} / \hspace{.1cm} \underbrace{U^T \hspace{.1cm} / \hspace{.1cm} L^TZ_i^T(y-X_i\beta)}_{\texttt{storage_q}}\big]$$

And from the loglikelihood, we already computed $U^T \hspace{.1cm} / \hspace{.1cm} L^TZ_i^T(y-X_i\beta)$. The strategy I deploy is calculating $X_i^T(y-X_i\beta)$, then solving $U \hspace{.1cm} / \hspace{.1cm} \cdots$ and left multiplying by $X^TZ_iL$. Finally I add and divide the whole thing by $\frac{1}{\sigma^2}$. 

$$\nabla_{\sigma^2} = - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i$$
$$ = -\frac{1}{2} \big[\text{Tr}\big(\Omega_i^{-1}\big) - r_i^T\Omega_i^{-1}\Omega_i^{-1}r_i\big]$$
$$ = -\frac{1}{2} \big[\text{Tr}\big(\frac{1}{\sigma^2}(I_{n_i} - Z_iLM^{-1}L^TZ_i^T)\big) - r_i^T[\frac{1}{\sigma^2}(I_{n_i} - Z_iLM^{-1}L^TZ_i^T)][\frac{1}{\sigma^2}(I_{n_i} - Z_iLM^{-1}L^TZ_i^T)]r_i$$
$$ = -\frac{1}{2} \big[\frac{n}{\sigma^2} - \frac{1}{\sigma^2}\text{Tr}\big(LM^{-1}L^TZ_i^TZ_i\big) - \frac{1}{(\sigma^2)^2}\big[\underbrace{r_i^Tr_i}_{\texttt{rtr}} - 2\underbrace{r_i^TZ_iLM^{-1}L^TZ_i^Tr_i}_{\texttt{qf}} + r_i^TZ_iLM^{-1}L^TZ_i^TZ_iLU / \underbrace{U^T / L^TZ_i^Tr_i}_{\texttt{storage_q}} \big]\big]$$
$$ = -\frac{1}{2\sigma^2} \big[n - \text{Tr}\big(LM^{-1}L^TZ_i^TZ_i\big) - \frac{1}{\sigma^2}\big[\texttt{rtr} - 2\texttt{qf} + r_i^TZ_iLM^{-1}L^TZ_i^TZ_iLU / \texttt{storage_q} \big]\big]$$

Calculating $LM^{-1}L^TZ_i^TZ_i$ for the trace is beneficial because it is much smaller than the original matrix and we will use it to calculate $\frac{\partial}{\partial \Sigma}$

$$\frac{\partial}{\partial \mathbf{\Sigma}} = - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i $$
$$ = -\frac{1}{\sigma^2}\big[Z_i^TZ_i - Z_i^TZ_iLM^{-1}L^TZ_i^TZ_i\big] + Z_i^T\Omega^{-1}r_i (Z_i^T\Omega^{-1}r_i)^T $$ 
Where $Z_i^t\Omega^{-1}r_i = \frac{1}{\sigma^2} [Z_i^Tr_i - Z_i^TZ_iLM⁻¹L^TZ_i^Tr_i]$ 


In [3]:
### try to work with smaller vectors instead of storage_n
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # Lt * ZtZ
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # LtZtZ * L
    
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ² # add sigma squared to diagonal aka + σ² * I
    end
    
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # computes and overwrites storage_qq with cholesky of M
    
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # storage_q = Zt(Y-Xβ)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # storage_q = Lt * Zt(Y-Xβ) upper triangle multiply and 
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # triangle solve storage_q = U \ LtZt(Y-Xβ) 
    # tri_solve = copy(obs.storage_q) # just checking that it doens't get overwritten, we good. 
    # but this is only one half (we have to do it again for U^T) if we want the full thing. 
    
    # l2 norm of residual vector (y-xB)'(y-xB)
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term AKA = (y-xb)'ZLM^{-1}LtZ(y-xb)
    logl += (rtr - qf) / σ² # (y-xb)'Ω^{-1}(y-xb) / σ²
    logl /= -2 # (y-xb)'Ω^{-1}(y-xb) / 2σ²
    
    ###################
    # Evaluate gradient
    ###################    
    if needgrad  # aka if someone says true then we calculate the gradient 
        # TODO: fill ∇β, ∇L, ∇σ² by gradients
        # ∇β = Xt Ω^{−1} 𝑟 = 1/σ² [Xt(y-xb) - Xt * Z * L (U^T / storage_q)]
        # = 1/σ² [Xty - XtXb - Xt Z L (U^T / storage_q)]  break up residual into already computed parts
        
        copy!(obs.∇β, obs.xty) # put Xty into end vector 
        BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), obs.∇β) # ∇β = Xty - XtXb
        
        # now second half Xt * Z * L * (U^t / U / LtZt(y-xb))  (we have ztx already) 
        # U / LtZt(y-xb) = storage_q
        copy!(obs.storage_q2, obs.storage_q)
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q2) # storage_q2  = U^T / storage_q
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q2) # storage_q2 = L * storage_q2  
        BLAS.gemv!('T', T(-1), obs.ztx, obs.storage_q2, T(1), obs.∇β) # - xtz * obs.storage_q2 + ∇β
        obs.∇β .= (1 / σ²) .* obs.∇β # divide whole thing by σ²
        # LM⁻¹LtZtr = storage_q2
        
        # ∇σ² = -1/2[tr(Ω⁻¹) - rtΩ⁻¹Ω⁻¹r] 
        # rtΩ⁻¹Ω⁻¹r is partially calculated in log likelihood
        # rtΩ⁻¹Ω⁻¹r = (1/σ²)² * [rtr - 2rtZLM⁻¹LtZtr + rtZLM⁻¹LtZtZLM⁻¹LtZtr]
        # = (1/σ²)² * [rtr - 2qf + rtZLM⁻¹LtZtZLM⁻¹LtZtr] so we need to calculate the last term
        # have LM⁻¹LtZtr already therefore just need to do transpose(LM⁻¹LtZtr)(ZtZ)(LM⁻¹LtZtr)
        
        copy!(obs.storage_q, obs.storage_q2) # just put LM⁻¹LtZtr into storage_q
        BLAS.gemm!('N', 'N', T(1), obs.ztz, obs.storage_q2, T(0), obs.storage_q)  # ZtZ * LM⁻¹LtZtr
        obs.∇σ² .= rtr .- 2qf .+ dot(obs.storage_q2, obs.storage_q) # rtr - 2qf + rtZLM⁻¹Lt * ZtZLM⁻¹LtZtr
        obs.∇σ² .= (1 / σ²) .* obs.∇σ² # rtΩ⁻¹Ω⁻¹r * σ²
        
        # tr(Ω^{−1}) = tr(1/σ²(In - ZLM^{-1}LtZt)) = n/σ² - 1/σ² tr(ZLM^{-1}LtZt)
        # trace is cyclical so tr(ZLM^{-1}LtZt) = tr(LM^{-1}LtZtZ) which is much smaller (q x q)
        # want to calculate LM⁻¹LtZtZ because it is in trace and ∇L
        copy!(obs.ltztzl, obs.ztz)
        BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.ltztzl) # L * Z'Z
        LAPACK.potrs!('U', obs.storage_qq, obs.ltztzl) # solve M⁻¹L'Z'Z
        BLAS.trmm!('L', 'L', 'N', 'N', T(1), L, obs.ltztzl) # L * M⁻¹L'Z'Z
        
        # ∇σ² = -1/2σ² [n - trace(LM⁻¹LtZtZ) - 1/σ²(rtr - 2qf + rtZLM⁻¹LtZtZLM⁻¹LtZtr)]
        obs.∇σ² .= n .- tr(obs.ltztzl) .- obs.∇σ² 
        obs.∇σ² .= (-1 / 2σ²) .* obs.∇σ²
        
        
        # ∇Σ = ZtΩ⁻¹rrtΩ⁻¹Z − Z^TΩ⁻¹Z 
        # −ZtΩ⁻¹Z = -1/σ² [ZtZ - (ZtZ)(LM⁻¹LtZtZ)]
        copy!(obs.∇Σ, obs.ztz)
        BLAS.gemm!('N', 'N', T(-1), obs.ztz, obs.ltztzl, T(1), obs.∇Σ) # ZtZ - ZtZ * LM⁻¹LtZtZ
        obs.∇Σ .= (- 1 / σ²) .* obs.∇Σ # obs.∇Σ = −ZtΩ⁻¹Z
        
        # ZtΩ⁻¹rrtΩ⁻¹Z
        # ZtΩ⁻¹r = 1/σ² [Ztr - (ZtZ)*(LM⁻¹LtZtr)] 
        # recall storage_q = ZtZ * LM⁻¹LtZtr 
        copy!(obs.storage_q2, obs.zty)
        BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), obs.storage_q2) # Z'r = Z'y - Z'Xβ
        obs.storage_q .= obs.storage_q2 .- obs.storage_q  # Z'r - Z'ZLM⁻¹L'Z'r
        obs.storage_q .= (1 / σ²) .* obs.storage_q # ZtΩ⁻¹r is qx1 

        BLAS.ger!(1.0, obs.storage_q, obs.storage_q, obs.∇Σ) # (ZtΩ⁻¹r)*(rtΩ⁻¹Z) + (−ZtΩ⁻¹Z)
        
    end    
    ###################
    # Return
    ###################        
    return logl
end

logl! (generic function with 2 methods)

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in HW2.

In [6]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [7]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ;

logl = logl!(obs, β, L, σ², true) = -3261.917755918759
obs.∇β = [-1.1937579038485637, -20.06810413006264, -12.323942329522518, -5.174641191561894, 28.208475690990344]
obs.∇σ² = [5.501620540890977]
obs.∇Σ = [0.40732856643411974 -0.10370457194287114 0.9214023976867127; -0.1037045719429481 -0.9907411137746662 -0.021658311185019763; 0.9214023976866843 -0.02165831118500082 -0.5355581080217697]


You will lose all 20 points if following statement throws `AssertionError`.

In [8]:
@assert abs(logl - (-3261.9177559187597)) < 1e-8
@assert norm(obs.∇β - [-1.1937579038479573, -20.068104130062466, 
        -12.323942329522424, -5.174641191562253, 28.208475690990248]) < 1e-8
@assert abs(obs.∇σ²[1] - (5.501620540891395)) < 1e-8
@assert norm(obs.∇Σ - [0.40732856643442306 -0.10370457194285436 0.9214023976868783; 
        -0.10370457194285436 -0.9907411137746756 -0.02165831118509598; 
        0.9214023976868783 -0.02165831118509598 -0.5355581080215592]) < 1e-8


### Efficiency

Benchmark for evaluating objective only. This is what we did in HW2.

In [9]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     681.291 ns (0.00% GC)
  median time:      694.265 ns (0.00% GC)
  mean time:        700.385 ns (0.00% GC)
  maximum time:     1.352 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     151

Benchmark for objective + gradient evaluation.

In [10]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.788 μs (0.00% GC)
  median time:      1.863 μs (0.00% GC)
  mean time:        1.877 μs (0.00% GC)
  maximum time:     16.163 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

My median runt time is 2.1μs. You will get full credit (10 pts) if the median run time is within 10μs.

In [11]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

## Q3. LmmModel type

We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a `LmmModel` object is simply the sum of log-likelihood/gradient of individual data points. 

Now we have a VECTOR of observations instead of a single point 

In [12]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MathProgBase.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz2 :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr  = Vector{T}(undef, abs2(q))
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr, ztr2, xtx, ztz2)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇Σ, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    # obtain gradient wrt L: m.∇L = m.∇L * L   ie keep adding just like he does above
    if needgrad
       BLAS.trmm!('R', 'L', 'N', 'N', T(1), m.L, m.∇L)
    end
    logl
end

logl!

## Q4. (20 pts) Test data

Let's generate a fake longitudinal data set to test our algorithm.

In [13]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file `lmm_data.csv`. **Do not put this file in Git.** It takes 246.6MB storage.

In [14]:
(isfile("lmm_data.csv") && filesize("lmm_data.csv") == 244697920) || 
open("lmm_data.csv", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p-1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q-1)
        print(io, "Z" * string(j) * (j < q-1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:q-1
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

### Correctness

Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [15]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.834275229633783e6
lmm.∇β = [11.772120067025336, -1925.6905435286562, -590.4811426626925, 2112.4417700625045, 2261.4604838394553]
lmm.∇σ² = [911.874762056731]
lmm.∇L = [24.419901311354298 24.01192277275017 16.56376848155844; 30.999259003092725 -59.4019026615511 46.58331413472236; 23.424706030628062 51.02946390987342 12.038870789804925]


Test correctness. You will loss all 20 points if following code throws `AssertError`.

In [16]:
@assert abs(obj - (-2.8342752296337797e6)) < 1e-6
@assert norm(lmm.∇β - [11.772120067073923, -1925.690543528459, 
        -590.4811426628461, 2112.441770062488, 2261.4604838395016]) < 1e-6
@assert norm(lmm.∇L - [24.419901311377576 24.01192277273847 16.563768481574012; 
        30.99925900307824 -59.40190266157969 46.58331413470851; 
        23.424706030649975 51.029463909858194 12.038870789777398]) < 1e-6
@assert abs(lmm.∇σ²[1] - (911.8747620554551)) < 1e-6

### Efficiency

Test efficiency.

In [17]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.854 ms (0.00% GC)
  median time:      1.874 ms (0.00% GC)
  mean time:        1.908 ms (0.00% GC)
  maximum time:     2.767 ms (0.00% GC)
  --------------
  samples:          2616
  evals/sample:     1

My median run time is 2.122ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [18]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [19]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## Q5. (30 pts) Starting point

For numerical optimization, a good starting point is critical. Let's start $\boldsymbol{\beta}$ and $\sigma^2$ from the least sqaures solutions (ignoring intra-individual correlations)
\begin{eqnarray*}
\boldsymbol{\beta}^{(0)} &=& \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)^{-1} \left(\sum_i \mathbf{X}_i^T \mathbf{y}_i\right) \\
\sigma^{2(0)} &=& \frac{\sum_i \|\mathbf{r}_i^{(0)}\|_2^2}{\sum_i n_i} = \frac{\sum_i \|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}^{(0)}\|_2^2}{\sum_i n_i}.
\end{eqnarray*}
To get a reasonable starting point for $\boldsymbol{\Sigma} = \mathbf{L} \mathbf{L}^T$, we can minimize the least squares criterion (ignoring the noise variance component)
$$
    \text{minimize} \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Derive the minimizer $\boldsymbol{\Sigma}^{(0)}$ (10 pts). 

Frobenius Norm = the square root of the sum of the absolute squares of its elements. So $||A||_F^2 = \sum_i\sum_j |a_{ij}|^2$. In addition, $||A||_F^2 = \text{Trace}(AA^H)$ where $A^H$ is the conjugate transpose. 

$||r_ir_i^T - Z_i\Sigma Z_i^T||^2_F = \text{Trace}((r_ir_i^T - Z_i\Sigma Z_i^T)^T(r_ir_i^T - Z_i\Sigma Z_i^T)) = \text{Trace}(r_ir_i^Tr_ir_i^T - 2Z_i\Sigma Z_i^Tr_ir_i^T + Z_i\Sigma Z_i^TZ_i\Sigma Z_i^T)$

$= \text{Trace}(r_ir_i^Tr_ir_i^T) - \text{Trace}(2Z_i^Tr_ir_i^TZ_i\Sigma) + \text{Trace}(\Sigma Z_i^TZ_i \Sigma Z_i^TZ_i)$

We want to minimize that quantity, which means taking partial derivatives wrt to the variable ($\Sigma$) and setting equal to 0. Note that $\frac{\partial}{\partial \Sigma} \text{Trace}(A\Sigma) = A^T$ and $\frac{\partial}{\partial \Sigma} \text{Trace}(\Sigma A\Sigma B) = B^T\Sigma^T A^T + A^T \Sigma^T B^T$

$\Rightarrow \frac{\partial}{\partial \Sigma} ||r_ir_i^T - Z_i\Sigma Z_i^T||^2_F = 0 -2Z_i^Tr_ir_i^TZ_i + 2Z_i^TZ_i\Sigma Z_i^TZ_i$

Hence, minimizing $\sum_i ||r_ir_i^T - Z_i\Sigma Z_i^T||^2_F$ is solving $\sum_i -2Z_i^Tr_ir_i^TZ_i + 2Z_i^TZ_i\Sigma Z_i^TZ_i = 0$ 

$\rightarrow \sum_i Z_i^Tr_ir_i^TZ_i = \sum_i Z_i^TZ_i\Sigma Z_i^TZ_i$

In order to get an initial value for $\Sigma$, we need to solve for $\Sigma$, which we can do by taking the vec. Recall that $\text{vec}(ABC) = (C^T \otimes A)\text{vec}B$, hence 
$$\text{vec}\big(\sum_i Z_i^Tr_ir_i^TZ_i\big) = \text{vec}\big(\sum_i \underbrace{Z_i^TZ_i}_{A}\underbrace{\Sigma}_{B} \underbrace{Z_i^TZ_i}_{C}\big)$$
$$\sum_i \big(Z_i^Tr_i \otimes Z_i^Tr_i\big) = \sum_i \big(Z_i^TZ_i \otimes Z_i^TZ_i\big) \text{vec}(\Sigma)$$
$$\Rightarrow \text{vec}(\Sigma^{(0)}) = \big(\sum_i Z_i^TZ_i \otimes Z_i^TZ_i\big)^{-1}\sum_i \big(Z_i^Tr_i \otimes Z_i^Tr_i\big)$$

We implement this start point strategy in the function `init_ls()`.

In [20]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # TODO: fill m.β, m.L, m.σ² by LS estimates
    n = length(m.data)
    
    
    fill!(m.β, 0.0)
    fill!(m.L, 0.0)
    fill!(m.σ², 0.0)
    den_sum = T(0)
    
    # β need to calculate the sum of XitXi and Xityi so add them each time. 
    for i in 1:n
        m.xtx .= m.xtx .+ m.data[i].xtx
        m.xty .= m.xty .+ m.data[i].xty
        
        # need to add all yty to m.σ²
        m.σ²[1] += m.data[i].yty
        
        # keep track of denominator for σ²
        den_sum = den_sum + size(m.data[i].X, 1)
    end
    
    mul!(m.β, m.xtx \ I, m.xty) # solve inverse, multiply to xty and save to beta
   
    # σ² need to sum over all residuals and sum of all datapoints. 
    m.σ²[1] += dot(m.β, BLAS.gemv!('N', T(1), m.xtx, m.β, T(-2), m.xty))
    m.σ² ./= den_sum # divide by sum of n_i
    
    # Σ min = (sum ZtZ kron ZtZ)^{-1}*(sum Ztr kron Ztr)
    fill!(m.ztr2, 0.0)
    fill!(m.ztz2, 0.0)
    fill!(m.ztr, 0.0)
    # so I will calculate ZtZ kron ZtZ for each i and update for each person

    m.xtx .= T(0)
    m.xty .= T(0)
    obs.storage_qq .= T(0)
    for i in 1:length(m.data)
        obs = m.data[i]
        m.ztz2 .+= kron(obs.ztz, obs.ztz) # get Z'Z kron Z'Z
        
        # need to calculate ztr for each person first      
        BLAS.gemv!('N', T(1), obs.ztx, m.β, T(0), obs.storage_q) # Ztxβ
        BLAS.axpby!(T(1), obs.zty, T(-1), obs.storage_q) #  Ztr = Zty - Ztxβ
        
        m.ztr2 .+= kron(obs.storage_q, obs.storage_q) # Ztr kron Ztr 
    end

    LAPACK.potrf!('L', m.ztz2) # computes and overwrites with cholesky
    LAPACK.potrs!('L', m.ztz2, m.ztr2) # solve (ZtZ kron ZtZ)^{-1}*(Ztr kron Ztr)
    # LAPACK.posv!('L', m.ztz2, m.ztr2) # (ZtZ kron ZtZ)^{-1}*(Ztr kron Ztr)
    # now "unvec" which is reshape
    m.L .= reshape(m.ztr2, q, q)
    LAPACK.potrf!('L', m.L) # computes and overwrites with cholesky

    m
end

init_ls!

In [21]:
lmm = LmmModel(obsvec)
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
# @show size(lmm.ztr2)
# @show lmm.ztr2
@show lmm.L;

logl!(lmm) = -3.3529912271664594e6
lmm.β = [0.12814248060847544, 6.497945035096693, -3.5021387747701356, 1.0026609224782135, 5.002697971721907]
lmm.σ² = [5.70170988595954]
lmm.L = [1.442089600524507 0.055234925755674846 0.04021508338255691; 0.03830200684866265 1.0546394793870688 0.06543803114908463; 0.027886674564416906 0.0610349951874949 1.0014753586943541]


### Correctness

Your start points should have a log-likelihood larger than -3.352991e6 (10 pts). The points you get are

In [22]:
# this is the points you get
(logl!(lmm) >  -3.353e6) * 10

10

### Efficiency

The start point should be computed quickly. Otherwise there is no point using it as a starting point. You get full credit (10 pts) if the median run time is within 1ms.

In [23]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 
  memory estimate:  1.12 MiB
  allocs estimate:  8007
  --------------
  minimum time:     384.875 μs (0.00% GC)
  median time:      516.042 μs (0.00% GC)
  mean time:        597.677 μs (19.23% GC)
  maximum time:     14.459 ms (95.60% GC)
  --------------
  samples:          8322
  evals/sample:     1

In [24]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## Q6. NLP via MathProgBase.jl

We define the NLP problem using the modelling tool MathProgBase.jl. Start-up code is given below. Modify if necessary to accomodate your own code.

In [25]:
"""
    fit!(m::LmmModel, solver=Ipopt.IpoptSolver(print_level=5))

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m::LmmModel,
        solver=Ipopt.IpoptSolver(print_level=5)
    )
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    optm = MathProgBase.NonlinearModel(solver)
    # set lower bounds and upper bounds of parameters
    # diagonal entries of Cholesky factor L should be >= 0
    lb   = fill(-Inf, npar)
    ub   = fill( Inf, npar)
    offset = p + 1
    for j in 1:q, i in j:q
        i == j && (lb[offset] = 0)
        offset += 1
    end
    # σ² should be >= 0
    lb[end] = 0
    MathProgBase.loadproblem!(optm, npar, 0, lb, ub, Float64[], Float64[], :Max, m)
    # starting point
    par0 = zeros(npar)
    modelpar_to_optimpar!(par0, m)
    MathProgBase.setwarmstart!(optm, par0)
    # optimize
    MathProgBase.optimize!(optm)
    optstat = MathProgBase.status(optm)
    optstat == :Optimal || @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    optimpar_to_modelpar!(m, MathProgBase.getsolution(optm))
    logl!(m, true)
    m
end

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset   += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MathProgBase.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in [:Grad])
            error("Unsupported feature $feat")
        end
    end
end

MathProgBase.features_available(m::LmmModel) = [:Grad]

function MathProgBase.eval_f(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MathProgBase.eval_grad_f(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

MathProgBase.eval_g(m::LmmModel, g, par) = nothing
MathProgBase.jac_structure(m::LmmModel) = Int[], Int[]
MathProgBase.eval_jac_g(m::LmmModel, J, par) = nothing

## Q7. (20 pts) Test drive

Now we can run our NLP solver to compute the MLE. For grading purpose, we first use the `:LD_LBFGS` (limited-memory BFGS) algorithm in NLopt.jl here.

In [26]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

@time fit!(lmm, NLopt.NLoptSolver(algorithm =:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000));

println("objective value at solution: ", logl!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))

objective value at starting point: -3.3529912271664594e6

  0.601676 seconds (1.41 M allocations: 77.586 MiB, 2.44% gc time, 81.84% compilation time)
objective value at solution: -2.834264502365345e6

solution values:
lmm.β = [0.12388537837783552, 6.4983373084688845, -3.500513045122987, 1.001821979786551, 5.001953759134669]
lmm.σ² = [1.5023521283251702]
lmm.L * transpose(lmm.L) = [2.0684734074012767 0.05298548776562313 0.03270542467962338; 0.05298548776562313 1.1218121650879602 0.05601741940842158; 0.03270542467962338 0.05601741940842158 1.012045075975027]
gradient @ solution:
lmm.∇β = [-0.02076862485362628, -0.026328579871236713, -0.010670280017588851, -0.02573390057455427, -0.0515042935562362]
lmm.∇σ² = [-0.023120619402732245]
lmm.∇L = [0.029564234445700145 0.05924166207970372 0.005205834926730552; 0.07920760965309105 -0.01359965507218792 -0.02132915470149322; 0.006885944413642297 -0.021988023206651264 0.00944892680614832]
sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + abs2(norm(Lowe

0.07428917290028683

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [27]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.83427e6
# gradient at solution should be small enough
@assert sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L))))) < 0.1

### Efficiency

My median run time is 132.5ms. You get 10 points if your median time is within 1s(=1000ms).

In [28]:
bm_bfgs = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval = 10000))) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 
  memory estimate:  11.14 KiB
  allocs estimate:  234
  --------------
  minimum time:     102.437 ms (0.00% GC)
  median time:      103.190 ms (0.00% GC)
  mean time:        104.023 ms (0.00% GC)
  maximum time:     108.357 ms (0.00% GC)
  --------------
  samples:          48
  evals/sample:     1

In [29]:
# this is the points you get
clamp(1 / (median(bm_bfgs).time / 1e9) * 10, 0, 10)

10.0

## Q8. (10 pts) Gradient free vs gradient-based methods

Advantage of using a modelling tool such as MathProgBase.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solver works best. 

Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.

See this [page](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for the descriptions of algorithms in NLopt.

Documentation for the Ipopt can be found [here](https://coin-or.github.io/Ipopt/). 

In [18]:
# vector of solvers to compare
solvers = [
    # NLopt: gradient-based algorithms
    NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12,
        xtol_rel = 1e-12, xtol_abs = 1e-12,
        maxeval=10000),
    NLopt.NLoptSolver(algorithm=:LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # NLopt: gradient-free algorithms
    NLopt.NLoptSolver(algorithm=:LN_BOBYQA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # Ipopt
    Ipopt.IpoptSolver(print_level=0)
]
# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for (i, solver) in enumerate(solvers)
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [19]:
DataFrame(Runtime = runtime, Objective = objvals, Gradnorm = gradnrm)

,Runtime,Objective,Gradnorm
,Float64,Float64,Float64
1,0.10316,-2.83426e6,0.0742892
2,0.129087,-2.83426e6,0.00527495
3,0.0448093,-2.83539e6,31729.5
4,5.55828,-2.83426e6,1.2764e-5


## Q9. (10 pts) Compare with existing art

Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [ ]:
method  = ["257", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### Your approach

In [ ]:
bm_257 = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm=:LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000))) setup=(init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

### lme4 

In [ ]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.txt")
"""

In [ ]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

In [ ]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### MixedModels.jl



In [ ]:
testdata = CSV.File("lmm_data.txt", types = Dict(1=>String)) |> DataFrame!

In [ ]:
mj = fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

In [ ]:
display(bm_mm)
mj

### Summary

In [ ]:
DataFrame(method = method, runtime = runtime, logl = loglike)

## Q10. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by XXX fold. 